In [ ]:
import pandas as pd
import os

In [ ]:
file_paths = {
    "VDJdb_beta": vdjdb_beta_read_path,
    "McPAS_beta": mcpastcr_beta_read_path,
    "IEDB_beta": iedb_beta_read_path,
    "pMTnet_beta": pmtnet_beta_read_path,
    "VDJdb_paired": vdjdb_paired_read_path,
    "McPAS_paired": mcpastcr_paired_read_path,
    "IEDB_paired": iedb_paired_read_path
}

In [ ]:
# Alle Daten speichern
all_data = {}

# Daten einlesen und vorbereiten
for file_name, path in file_paths.items():
    if os.path.exists(path):
        try:
            df = pd.read_csv(path, sep=None, engine="python")

            # tcr_key erstellen
            if "TRA_CDR3" in df.columns:
                df["tcr_key"] = df["TRA_CDR3"].astype(str) + '_' + df["TRB_CDR3"]
            else:
                df["tcr_key"] = df["TRB_CDR3"]

            all_data[file_name] = df[["tcr_key", "Epitope"]].drop_duplicates()

            print(f"{file_name} geladen mit {len(df)} Einträgen.")

        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file_name}: {e}")
    else:
        print(f"Datei nicht gefunden: {file_name}")

# Analyse: Welcher Datensatz sollte im Testset sein?
for test_file_name, test_df in all_data.items():
    # Alle anderen Daten als Trainingsset
    train_df = pd.concat([data for name, data in all_data.items() if name != test_file_name]).drop_duplicates()

    # Erstelle Sets für schnelles Lookup
    seen_tcrs = set(train_df["tcr_key"])
    seen_epitopes = set(train_df["Epitope"])

    # TPP3-Definition: Weder TCR noch Epitope wurden im Trainingsset gesehen
    tpp3_pairs = test_df[~test_df["tcr_key"].isin(seen_tcrs) & ~test_df["Epitope"].isin(seen_epitopes)]

    # Ausgabe der Ergebnisse
    print(f"\n**Wenn {test_file_name} als Testset verwendet wird:**")
    print(f"  - TPP3-Paare im Testset: {len(tpp3_pairs)}")
    print(f"  - Gesamt Test-Paare: {len(test_df)}")

In [ ]:
import os
import pandas as pd

# Define file paths
beta_train_file = f'{pipeline_data_splitted}/{precision}/beta/train.tsv'
beta_validation_file = f'{pipeline_data_splitted}/{precision}/beta/validation.tsv'
vdjdb_test_file = vdjdb_beta_read_path  # Path to the original VDJdb data

# Load train and validation data
df_train = pd.read_csv(beta_train_file, sep='\t')
df_validation = pd.read_csv(beta_validation_file, sep='\t')

# Concatenate train and validation for analysis
trainval_df = pd.concat([df_train, df_validation], ignore_index=True)

# Load VDJdb dataset
vdjdb_df = pd.read_csv(vdjdb_test_file, sep='\t')

# Prepare the data by creating tcr_key and ensuring clean values
trainval_df["tcr_key"] = trainval_df["TRB_CDR3"].astype(str).str.strip()
trainval_df["Epitope"] = trainval_df["Epitope"].astype(str).str.strip()

vdjdb_df["tcr_key"] = vdjdb_df["TRB_CDR3"].astype(str).str.strip()
vdjdb_df["Epitope"] = vdjdb_df["Epitope"].astype(str).str.strip()

# Create lookup sets from train+validation data
seen_tcrs = set(trainval_df["tcr_key"])
seen_epitopes = set(trainval_df["Epitope"])

# Identify TPP3 pairs in VDJdb (where both TCR and Epitope are unseen)
tpp3_pairs = vdjdb_df[~vdjdb_df["tcr_key"].isin(seen_tcrs) & ~vdjdb_df["Epitope"].isin(seen_epitopes)]

# Print the result
print(f"\n**TPP3 Analysis for VDJdb in Train + Validation**")
print(f"  - TPP3-Paare im Testset: {len(tpp3_pairs)}")
print(f"  - Gesamt Test-Paare: {len(vdjdb_df)}")


In [ ]:
import pandas as pd
import os

# Function to classify tasks based on TCR and epitope presence
def calculate_task(row, known_epitopes, known_tcr, paired=False):
    if paired:
        tra_cdr3 = str(row['TRA_CDR3']) if pd.notna(row['TRA_CDR3']) else ''
        trb_cdr3 = str(row['TRB_CDR3']) if pd.notna(row['TRB_CDR3']) else ''
        tcr = tra_cdr3 + '_' + trb_cdr3
    else:
        tcr = row['TRB_CDR3']
    
    epitope_exists = row['Epitope'] in known_epitopes
    cdr3_exists = tcr in known_tcr
    
    if epitope_exists and cdr3_exists:
        return 'TPP1'
    elif epitope_exists and not cdr3_exists:
        return 'TPP2'
    elif not epitope_exists and not cdr3_exists:
        return 'TPP3'
    elif not epitope_exists and cdr3_exists:
        return 'TPP4'
    raise Exception("Something seems wrong")

# Placeholder for the data
all_data = {}

# Load and prepare data
for file_name, path in file_paths.items():
    if os.path.exists(path):
        try:
            df = pd.read_csv(path, sep=None, engine="python")

            # Create tcr_key
            if "TRA_CDR3" in df.columns:
                paired = True
                df["tcr_key"] = df["TRA_CDR3"].astype(str) + '_' + df["TRB_CDR3"]
            else:
                paired = False
                df["tcr_key"] = df["TRB_CDR3"]

            all_data[file_name] = df
            print(f"{file_name} geladen mit {len(df)} Einträgen.")

        except Exception as e:
            print(f"Fehler beim Verarbeiten der Datei {file_name}: {e}")
    else:
        print(f"Datei nicht gefunden: {file_name}")

# Analyse: Classify TPP tasks
for test_file_name, test_df in all_data.items():
    # Define training data (excluding the current test set)
    train_df = pd.concat([data for name, data in all_data.items() if name != test_file_name]).drop_duplicates()

    seen_tcrs = set(train_df["tcr_key"])
    seen_epitopes = set(train_df["Epitope"])

    # Determine if it's a paired dataset
    paired = "TRA_CDR3" in test_df.columns

    # Apply classification
    test_df['task'] = test_df.apply(lambda row: calculate_task(row, seen_epitopes, seen_tcrs, paired=paired), axis=1)

    # Count TPP3 pairs
    tpp3_pairs = (test_df['task'] == 'TPP3').sum()

    # Print the results
    print(f"\n**Wenn {test_file_name} als Testset verwendet wird:**")
    print(f"  - TPP3-Paare im Testset: {tpp3_pairs}")
    print(f"  - Gesamt Test-Paare: {len(test_df)}")


In [ ]:
import pandas as pd
import os

# Function to classify TPP tasks based on TCR and epitope presence
def calculate_task(row, known_epitopes, known_tcr, paired=False):
    if paired:
        tra_cdr3 = str(row['TRA_CDR3']) if pd.notna(row['TRA_CDR3']) else ''
        trb_cdr3 = str(row['TRB_CDR3']) if pd.notna(row['TRB_CDR3']) else ''
        tcr = tra_cdr3 + '_' + trb_cdr3
    else:
        tcr = row['TRB_CDR3']
    
    epitope_exists = row['Epitope'] in known_epitopes
    cdr3_exists = tcr in known_tcr
    
    if epitope_exists and cdr3_exists:
        return 'TPP1'
    elif epitope_exists and not cdr3_exists:
        return 'TPP2'
    elif not epitope_exists and not cdr3_exists:
        return 'TPP3'
    elif not epitope_exists and cdr3_exists:
        return 'TPP4'
    raise Exception("Something seems wrong")


# Load train and validation data
train_file = f'{pipeline_data_splitted}/{precision}/beta/train.tsv'
validation_file = f'{pipeline_data_splitted}/{precision}/beta/validation.tsv'
vdjdb_test_file = vdjdb_beta_read_path  # Path to the VDJdb test dataset

df_train = pd.read_csv(train_file, sep='\t')
df_validation = pd.read_csv(validation_file, sep='\t')

# Combine train and validation datasets
trainval_df = pd.concat([df_train, df_validation], ignore_index=True)

# Load VDJdb test data
vdjdb_df = pd.read_csv(vdjdb_test_file, sep='\t')

# Create tcr_key and clean data
trainval_df["tcr_key"] = trainval_df["TRB_CDR3"].astype(str).str.strip()
trainval_df["Epitope"] = trainval_df["Epitope"].astype(str).str.strip()

vdjdb_df["tcr_key"] = vdjdb_df["TRB_CDR3"].astype(str).str.strip()
vdjdb_df["Epitope"] = vdjdb_df["Epitope"].astype(str).str.strip()

# Generate lookup sets for fast comparison
seen_tcrs = set(trainval_df["tcr_key"])
seen_epitopes = set(trainval_df["Epitope"])

# Apply classification to the VDJdb dataset
vdjdb_df['task'] = vdjdb_df.apply(lambda row: calculate_task(row, seen_epitopes, seen_tcrs, paired=False), axis=1)

# Count TPP3 pairs
tpp3_pairs = (vdjdb_df['task'] == 'TPP3').sum()

# Print the results
print(f"\n**TPP Analysis for VDJdb with Train + Validation**")
print(f"  - TPP3-Paare im Testset: {tpp3_pairs}")
print(f"  - Gesamt Test-Paare: {len(vdjdb_df)}")


In [ ]:
# Original TPP3 Paare vor dem Hinzufügen negativer Daten
original_tpp3 = vdjdb_df[(vdjdb_df['task'] == 'TPP3')][["tcr_key", "Epitope"]]

# Prüfen, ob diese TPP3-Paare in den negativen Daten des Train/Validation-Sets auftauchen
negative_in_trainval = trainval_df[trainval_df['Binding'] == 0][["tcr_key", "Epitope"]]

# Vergleichen
tpp3_now_seen = original_tpp3.merge(negative_in_trainval, on=["tcr_key", "Epitope"], how="inner")

print(f"Anzahl der ursprünglichen TPP3-Paare, die jetzt in negativen Daten des Train/Validation-Sets vorkommen: {len(tpp3_now_seen)}")

# Zeige einige Beispiele
if not tpp3_now_seen.empty:
    print(tpp3_now_seen.head(10))
else:
    print("Keine der ursprünglichen TPP3-Paare wurden in den negativen Daten gefunden.")

# Prüfen, ob die ursprünglichen TPP3-Paare in den positiven Daten des Train/Validation-Sets enthalten sind
positive_in_trainval = trainval_df[trainval_df['Binding'] == 1][["tcr_key", "Epitope"]]

# Vergleich durchführen
tpp3_in_pos_trainval = original_tpp3.merge(positive_in_trainval, on=["tcr_key", "Epitope"], how="inner")

print(f"Anzahl der ursprünglichen TPP3-Paare, die jetzt in den positiven Train/Validation-Daten vorkommen: {len(tpp3_in_pos_trainval)}")

# Vergleiche, ob die TCR-Keys in den ursprünglichen TPP3-Paaren mit denen im Testset übereinstimmen
missing_tcr_keys = original_tpp3[~original_tpp3["tcr_key"].isin(test_df["tcr_key"])]

print(f"Anzahl der TPP3-Paare, deren TCR-Key im Testset fehlt: {len(missing_tcr_keys)}")